In [1]:
%pip install pandas selenium

In [7]:
import pandas as pd
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.action_chains import ActionChains
import time
import re
import sqlite3
genres = ["Action", "Adventure", "Animation", "Comedy", "Crime"]
links = ["https://www.imdb.com/search/title/?title_type=feature&genres=action&release_date=2024-01-01,2024-12-31",
    "https://www.imdb.com/search/title/?title_type=feature&genres=adventure&release_date=2024-01-01,2024-12-31",
    "https://www.imdb.com/search/title/?title_type=feature&genres=animation&release_date=2024-01-01,2024-12-31",
    "https://www.imdb.com/search/title/?title_type=feature&genres=comedy&release_date=2024-01-01,2024-12-31",
    "https://www.imdb.com/search/title/?title_type=feature&genres=crime&release_date=2024-01-01,2024-12-31"]
try:
    driver = webdriver.Chrome()
    print("Crome Web-driver created Successfully")
except Exception as e:
    print("ERROR: ",e)      
try:
    conn = sqlite3.connect('IMDB.db')
    cursor = conn.cursor()
    print("Database created Successfully")
    cursor.execute('''CREATE TABLE IF NOT EXISTS Movies (Title TEXT,Duration FLOAT,Rating FLOAT,Votes INT,Genre TEXT)''')
    print("Database Table create successfully")
except Exception as e:
    print("ERROR: ",e)

def scrapeData(url):
    movie_dict = {'Title': [], 'Duration': [], 'Rating': [], 'Votes': []} 
    driver.get(url)
    print("URL link send to web driver")
    time.sleep(3)
    print("clicking '50 more' buttons")  
    while True:
        try:
            load_more = driver.find_element(By.XPATH, '//*[@id="__next"]/main/div[2]/div[3]/section/section/div/section/section/div[2]/div/section/div[2]/div[2]/div[2]/div/span/button')
            ActionChains(driver).move_to_element(load_more).perform()
            load_more.click()
            time.sleep(3)
        except Exception as e:
            print("click '50 more' buttons stopped")
            break
    movies_item = driver.find_elements(By.XPATH, '//*[@id="__next"]/main/div[2]/div[3]/section/section/div/section/section/div[2]/div/section/div[2]/div[2]/ul/li')
    print("getting movies details")
    for movies in movies_item:
        try:
            title = movies.find_element(By.XPATH, "./div/div/div/div[1]/div[2]/div[1]/a/h3").text
            duration = movies.find_element(By.XPATH, "./div/div/div/div[1]/div[2]/div[2]/span[2]").text
            rating  = movies.find_element(By.XPATH, "./div/div/div/div[1]/div[2]/span/div/span/span[1]").text
            voting = movies.find_element(By.XPATH, "./div/div/div/div[1]/div[2]/span/div/span/span[2]").text
            movie_dict['Title'].append(title)
            movie_dict['Rating'].append(rating)
            movie_dict['Votes'].append(voting)
            movie_dict['Duration'].append(duration)
        except Exception as e:
            pass
    return movie_dict
def main():
    if driver is None: 
        print("Crome Web-driver problem")
        return
    for i, link in enumerate(links):
        print(f"start getting {genres[i]} movies details")
        data = scrapeData(link)
        df = pd.DataFrame(data)
        df['Genre'] = genres[i]
        df.to_csv(f"{genres[i]}.csv", index=False)
        print(f"{genres[i]} Movie details stored in {genres[i]}.csv file successfully")
    driver.quit()
def changeTitle(title):
    if pd.isna(title):
        return None
    title = re.sub(r'^\d+\.\s*', '', title) 
    return title.strip()
def changeVotes(vote):
    if pd.isna(vote): 
        return 0
    if isinstance(vote, str):
        vote = vote.strip("() ").replace(",", "")  
        if vote.isdigit():  
            return int(vote)
        multiplier = 1
        if "K" in vote:
            multiplier = 1000
            vote = vote.replace("K", "")
        elif "M" in vote:
            multiplier = 1000000
            vote = vote.replace("M", "")
        elif "B" in vote:
            multiplier = 1000000000
            vote = vote.replace("B", "")
        return int(float(vote) * multiplier) 
def changeDuration(duration):  
    if pd.isna(duration): 
        return 0.0
    hours, minutes = 0, 0
    if "h" in duration:
        hours = int(duration.split("h")[0].strip())  
    if "m" in duration:
        minutes = int(duration.split("h")[-1].replace("m", "").strip())  
    return float(f"{hours}.{minutes:02d}")
def cleanData():
    for i in genres:
        a = pd.read_csv(f"{i}.csv")
        a["Title"] = a["Title"].apply(changeTitle).astype("str").str.replace("'", "") 
        a["Duration"] = a["Duration"].apply(changeDuration)
        a["Rating"] = pd.to_numeric(a["Rating"], errors="coerce").fillna(0.0).astype("float64")
        a["Votes"] = a["Votes"].apply(changeVotes).astype("Int64")
        a["Genre"] = a["Genre"].astype("str")
        a.drop_duplicates(subset=['Title', 'Duration', 'Rating','Votes','Genre'], keep='first', inplace=True) 
        a = a[(a != 0).all(axis=1)] 
        a.to_csv(f"{i}_cl.csv", index=False)
        print(f"cleaned data stored  in {i}_cl.csv successfully")
def mergeData():
    d = [pd.read_csv(f"{i}_cl.csv") for i in genres] 
    data = pd.concat(d, ignore_index=True)  
    data.drop_duplicates(subset=['Title', 'Duration', 'Rating', 'Votes', 'Genre'], keep='first', inplace=True) 
    data.to_csv("final.csv", index=False)  
    print("final.csv was created successfully")
    return data  
def storeData(data):   
    try:     
        cursor.executemany("INSERT INTO Movies (Title, Duration, Rating, Votes, Genre) VALUES (?, ?, ?, ?, ?)", data.itertuples(index=False, name=None))
        conn.commit()
        print("data successfully inserted into database")
    except Exception as e:
        print("Error: ",e)
    a = cursor.execute('''select * from Movies LIMIT 5''')
    for i in a:
        print(i)
    conn.close()
main()
cleanData()
a= mergeData()
storeData(a)

Crome Web-driver created Successfully
Database created Successfully
Database Table create successfully
start getting Action movies details
URL link send to web driver
clicking '50 more' buttons
click '50 more' buttons stopped
getting movies details
Action Movie details stored in Action.csv file successfully
start getting Adventure movies details


WebDriverException: Message: unknown error: net::ERR_ADDRESS_UNREACHABLE
  (Session info: chrome=134.0.6998.89)
Stacktrace:
	GetHandleVerifier [0x00007FF6BE72FE65+26629]
	(No symbol) [0x00007FF6BE696030]
	(No symbol) [0x00007FF6BE52931A]
	(No symbol) [0x00007FF6BE525D10]
	(No symbol) [0x00007FF6BE5168C9]
	(No symbol) [0x00007FF6BE518638]
	(No symbol) [0x00007FF6BE516BD6]
	(No symbol) [0x00007FF6BE516656]
	(No symbol) [0x00007FF6BE51631A]
	(No symbol) [0x00007FF6BE513E9F]
	(No symbol) [0x00007FF6BE51478C]
	(No symbol) [0x00007FF6BE52D16A]
	(No symbol) [0x00007FF6BE5D102E]
	(No symbol) [0x00007FF6BE5A7AAA]
	(No symbol) [0x00007FF6BE5D0169]
	(No symbol) [0x00007FF6BE5A7883]
	(No symbol) [0x00007FF6BE570550]
	(No symbol) [0x00007FF6BE571803]
	GetHandleVerifier [0x00007FF6BEA872DD+3529853]
	GetHandleVerifier [0x00007FF6BEA9DA42+3621858]
	GetHandleVerifier [0x00007FF6BEA924F3+3575443]
	GetHandleVerifier [0x00007FF6BE7FB79A+860474]
	(No symbol) [0x00007FF6BE6A08AF]
	(No symbol) [0x00007FF6BE6